This tutorial was written by [Won Hee Lee](https://wonhee-lee.github.io/) for SWCON425.

This current version has been created as a Jupyter notebook with Python3 for SWCON425, Data Science and Visualization.

# Pandas 2

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

sns.set()
sns.set_context('talk')
np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.options.display.max_rows = 7
pd.options.display.max_columns = 8
pd.set_option('display.precision', 2)
# This option stops scientific notation for pandas
# pd.set_option('display.float_format', '{:.2f}'.format)

## Case Study: What was the most popular name in New York last year (2019)?

Now let's play around with the large baby names dataset. We'll start by loading that dataset from the social security administration's website.

https://www.ssa.gov/OACT/babynames/index.html

https://www.ssa.gov/data

We can download data from the internet with Python, and do so only if needed:

In [2]:
import requests
from pathlib import Path

namesbystate_path = Path('namesbystate.zip')
data_url = 'https://www.ssa.gov/oact/babynames/state/namesbystate.zip'

if not namesbystate_path.exists():
    print('Downloading...', end=' ')
    resp = requests.get(data_url)
    with namesbystate_path.open('wb') as f:
        f.write(resp.content)
    print('Done!')

Let's use Python to understand how this data is laid out:

In [3]:
import zipfile
zf = zipfile.ZipFile(namesbystate_path, 'r')
print([f.filename for f in zf.filelist])

['AK.TXT', 'AL.TXT', 'AR.TXT', 'AZ.TXT', 'CA.TXT', 'CO.TXT', 'CT.TXT', 'DC.TXT', 'DE.TXT', 'FL.TXT', 'GA.TXT', 'HI.TXT', 'IA.TXT', 'ID.TXT', 'IL.TXT', 'IN.TXT', 'KS.TXT', 'KY.TXT', 'LA.TXT', 'MA.TXT', 'MD.TXT', 'ME.TXT', 'MI.TXT', 'MN.TXT', 'MO.TXT', 'MS.TXT', 'MT.TXT', 'NC.TXT', 'ND.TXT', 'NE.TXT', 'NH.TXT', 'NJ.TXT', 'NM.TXT', 'NV.TXT', 'NY.TXT', 'OH.TXT', 'OK.TXT', 'OR.TXT', 'PA.TXT', 'RI.TXT', 'SC.TXT', 'SD.TXT', 'StateReadMe.pdf', 'TN.TXT', 'TX.TXT', 'UT.TXT', 'VA.TXT', 'VT.TXT', 'WA.TXT', 'WI.TXT', 'WV.TXT', 'WY.TXT']


To keep the data small enough to avoid crashing datahub, we're going to look at only New York rather than looking at the national dataset.

Let's have a look at the New York data, it should give us an idea about the structure of the whole thing:

In [4]:
ny_name = 'NY.TXT'
with zf.open(ny_name) as f:
    for i in range(10):
        print(f.readline().rstrip().decode())

NY,F,1910,Mary,1923
NY,F,1910,Helen,1290
NY,F,1910,Rose,990
NY,F,1910,Anna,951
NY,F,1910,Margaret,926
NY,F,1910,Dorothy,897
NY,F,1910,Ruth,713
NY,F,1910,Lillian,648
NY,F,1910,Florence,604
NY,F,1910,Frances,589


This is equivalent (on macOS or Linux) to extracting the full `NY.TXT` file to disk and then using the `head` command (if you're on Windows, don't try to run the cell below):

In [5]:
zf.extract(ny_name)
!head {ny_name}

NY,F,1910,Mary,1923
NY,F,1910,Helen,1290
NY,F,1910,Rose,990
NY,F,1910,Anna,951
NY,F,1910,Margaret,926
NY,F,1910,Dorothy,897
NY,F,1910,Ruth,713
NY,F,1910,Lillian,648
NY,F,1910,Florence,604
NY,F,1910,Frances,589


In [6]:
!echo {ny_name}

NY.TXT


A couple of practical comments:

* The above is using special tricks in IPython that let you call operating system commands via `!cmd`, and that expand Python variables in such commands with the `{var}` syntax. You can find more about IPython's special tricks [in this tutorial](https://github.com/ipython/ipython-in-depth/blob/master/examples/IPython%20Kernel/Beyond%20Plain%20Python.ipynb).

* `head` doesn't work on Windows, though there are equivalent Windows commands. But by using Python code, even if it's a little bit more verbose, we have a 100% portable solution.

* If the `NY.TXT` file was huge, it would be wasteful to write it all to disk only to look at the start of the file.

The last point is an important, and general theme of this course: we need to learn how to operate with data only on an as-needed basis, because there are many situations in the real world where we can't afford to brute-force 'download all the things'.

Let's remove the `NY.TXT` file to make sure we keep working with our compressed data, as if we couldn't extract it:

In [7]:
#import os; os.unlink(ny_name)

## Question 1: What was the most popular baby name in NY last year?

In [8]:
import pandas as pd

field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ny_name) as fh:
    ny = pd.read_csv(fh, header=None, names=field_names)
ny.head()

,State,Sex,Year,Name,Count
0,NY,F,1910,Mary,1923
1,NY,F,1910,Helen,1290
2,NY,F,1910,Rose,990
3,NY,F,1910,Anna,951
4,NY,F,1910,Margaret,926


### Indexing Review

Let's play around a bit with our indexing techniques from last lecture.

In [9]:
# print first five 'Count' using .head()

ny['Count'].head()

0    1923
1    1290
2     990
3     951
4     926
Name: Count, dtype: int64

In [10]:
# print first three rows

ny[0:3]

,State,Sex,Year,Name,Count
0,NY,F,1910,Mary,1923
1,NY,F,1910,Helen,1290
2,NY,F,1910,Rose,990


In [11]:
# print first three rows' 'Name' and 'Count' using .iloc

ny.iloc[0:3, [3, 4]]

,Name,Count
0,Mary,1923
1,Helen,1290
2,Rose,990


In [12]:
# print first four rows' 'State' using .loc

ny.loc[0:3, 'State']

0    NY
1    NY
2    NY
3    NY
Name: State, dtype: object

In [13]:
# print first five rows' 'Name' using .head

ny['Name'].head()

0        Mary
1       Helen
2        Rose
3        Anna
4    Margaret
Name: Name, dtype: object

In [14]:
# print first five rows' 'Name' using .head as a dataframe

#ny[['Name']].head()
ny['Name'].to_frame().head()

,Name
0,Mary
1,Helen
2,Rose
3,Anna
4,Margaret


In [15]:
# print last five rows' that 'Year' is 2017

ny[ny['Year'] == 2017].tail()

,State,Sex,Year,Name,Count
301434,NY,M,2017,Zephaniah,5
301435,NY,M,2017,Zidan,5
301436,NY,M,2017,Ziyad,5
301437,NY,M,2017,Zohan,5
301438,NY,M,2017,Zubair,5


## Understanding the Data

In [16]:
ny.head()

,State,Sex,Year,Name,Count
0,NY,F,1910,Mary,1923
1,NY,F,1910,Helen,1290
2,NY,F,1910,Rose,990
3,NY,F,1910,Anna,951
4,NY,F,1910,Margaret,926


We can get a sense for the shape of our data:

In [17]:
ny.shape

(309532, 5)

In [18]:
ny.size

1547660

Pandas will give us a summary overview of the *numerical* data in the DataFrame:

In [19]:
ny.describe()

,Year,Count
count,309532.00,309532.00
mean,1979.59,81.31
std,29.69,323.54
...,...,...
50%,1985.00,13.00
75%,2005.00,38.00
max,2021.00,10026.00


And let's look at the *structure* of the DataFrame:

In [20]:
ny.index

RangeIndex(start=0, stop=309532, step=1)

### Sorting

What we've done so far is NOT exploratory data analysis. We were just playing around a bit with the capabilities of the pandas library. Now that we're done, let's turn to the problem at hand: Identifying the most common name in New York last year.

In [21]:
# sort ny2017 by 'Count' and print first ten rows to ny_sorted

ny2017 = ny[ny['Year'] == 2017]
ny_sorted = ny2017.sort_values(by='Count', ascending=False)
ny_sorted.head(10)

,State,Sex,Year,Name,Count
299402,NY,M,2017,Liam,1428
299403,NY,M,2017,Noah,1241
164229,NY,F,2017,Olivia,1230
...,...,...,...,...,...
164231,NY,F,2017,Sophia,1008
164232,NY,F,2017,Mia,1001
299407,NY,M,2017,Michael,997


In [22]:
# write a single line of code that works the same as '1-8'

ny[ny['Year'] == 2017].sort_values(by='Count', ascending=False).head(10)    

,State,Sex,Year,Name,Count
299402,NY,M,2017,Liam,1428
299403,NY,M,2017,Noah,1241
164229,NY,F,2017,Olivia,1230
...,...,...,...,...,...
164231,NY,F,2017,Sophia,1008
164232,NY,F,2017,Mia,1001
299407,NY,M,2017,Michael,997


## Question 2:  Find baby names that start with j

### Approach 1: Use list comprehensions

In [23]:
babynames = ny
babynames

,State,Sex,Year,Name,Count
0,NY,F,1910,Mary,1923
1,NY,F,1910,Helen,1290
2,NY,F,1910,Rose,990
...,...,...,...,...,...
309529,NY,M,2021,Zaidan,5
309530,NY,M,2021,Zaine,5
309531,NY,M,2021,Zymir,5


In [24]:
babynames["Name"].head(10)

0        Mary
1       Helen
2        Rose
       ...   
7     Lillian
8    Florence
9     Frances
Name: Name, Length: 10, dtype: object

In [25]:
# Find names that start with 'J' and save to boolean array

starts_with_j = [x[0] == 'J' for x in babynames["Name"]]
babynames[starts_with_j].head(5)

,State,Sex,Year,Name,Count
14,NY,F,1910,Josephine,431
29,NY,F,1910,Jean,250
30,NY,F,1910,Julia,245
44,NY,F,1910,Jennie,178
84,NY,F,1910,Jane,84


In [26]:
# Find names that start with 'J'

j_names = babynames[[x[0]=='J' for x in babynames['Name']]]
j_names

,State,Sex,Year,Name,Count
14,NY,F,1910,Josephine,431
29,NY,F,1910,Jean,250
30,NY,F,1910,Julia,245
...,...,...,...,...,...
309389,NY,M,2021,Jones,5
309390,NY,M,2021,Jovan,5
309391,NY,M,2021,Jupiter,5


### Approach 2: Use the Series.str methods.

In [27]:
babynames["Name"].str.startswith('J').head(10)

0    False
1    False
2    False
     ...  
7    False
8    False
9    False
Name: Name, Length: 10, dtype: bool

In [28]:
# Find names that start with 'J' using Series.str.startswith() method and save to boolean array¶

starts_with_j = babynames["Name"].str.startswith('J')
starts_with_j.head(10)

0    False
1    False
2    False
     ...  
7    False
8    False
9    False
Name: Name, Length: 10, dtype: bool

In [29]:
# Find 5 random names that start with 'J' using Series.str.startswith() methods

babynames[babynames["Name"].str.startswith('J')].sample(5)

,State,Sex,Year,Name,Count
141560,NY,F,2008,Johanna,41
147569,NY,F,2010,Jaiden,12
269621,NY,M,2002,Jakub,14
30883,NY,F,1949,Jocelyn,12
35032,NY,F,1953,Jan,139


In [30]:
# Find 5 random names that contain 'ad' using Series.str.contains() method

babynames[babynames["Name"].str.contains('ad')].sample(5)

,State,Sex,Year,Name,Count
153564,NY,F,2012,Gladys,6
257133,NY,M,1995,Jade,9
128394,NY,F,2002,Maddie,5
198546,NY,M,1945,Conrad,21
153984,NY,F,2012,Mairead,5


In [31]:
# Split names that contain 'a' using Series.str.split() method and print with dataframe

babynames['Name'].str.split('a').to_frame().head(5)

,Name
0,"[M, ry]"
1,[Helen]
2,[Rose]
3,"[Ann, ]"
4,"[M, rg, ret]"


## Question 3: Sort names by their length

Suppose we want to sort all baby names in New York by their length.

In [32]:
babynames.iloc[[i for i, m in sorted(enumerate(babynames['Name']), key=lambda x: -len(x[1]))]].head(5)

,State,Sex,Year,Name,Count
250865,NY,M,1991,Michaelanthony,5
4810,NY,F,1917,Maryelizabeth,5
9310,NY,F,1923,Maryelizabeth,5
45464,NY,F,1960,Maryelizabeth,6
48458,NY,F,1962,Maryelizabeth,6


### Approach 1: Create a temporary column

Create a new series of only the lengths. Then add that series to the dataframe as a column. Then sort by that column. Then drop that column.

In [33]:
# create a new series of only the lengths
babyname_lengths = babynames['Name'].str.len()

#add that series to the dataframe as a column

babynames['Name Length'] = babyname_lengths
babynames.head(5)

,State,Sex,Year,Name,Count,Name Length
0,NY,F,1910,Mary,1923,4
1,NY,F,1910,Helen,1290,5
2,NY,F,1910,Rose,990,4
3,NY,F,1910,Anna,951,4
4,NY,F,1910,Margaret,926,8


In [34]:
#sort by the temporary column
babynames = babynames.sort_values(by='Name Length', ascending=False)
babynames.head(5)

,State,Sex,Year,Name,Count,Name Length
250865,NY,M,1991,Michaelanthony,5,14
50131,NY,F,1963,Maryelizabeth,5,13
70613,NY,F,1975,Maryelizabeth,5,13
102299,NY,F,1991,Samanthamarie,6,13
113868,NY,F,1996,Maryelizabeth,6,13


In [35]:
#drop the temporary column
babynames = babynames.drop(columns=['Name Length'])
babynames.head(5)

,State,Sex,Year,Name,Count
250865,NY,M,1991,Michaelanthony,5
50131,NY,F,1963,Maryelizabeth,5
70613,NY,F,1975,Maryelizabeth,5
102299,NY,F,1991,Samanthamarie,6
113868,NY,F,1996,Maryelizabeth,6


We can also use the Python `map` function if we want to use an arbitrarily defined function. Suppose we want to sort by the number of occurrences of "dr" plus the number of occurences of "ea".

In [36]:
def dr_ea_count(string):
    return string.count('dr') + string.count('ea')

#create the temporary column
babynames["dr_ea_count"] = babynames["Name"].map(dr_ea_count)

#sort by the temporary column
babynames = babynames.sort_values(by = "dr_ea_count", ascending=False)

#drop that column
babynames = babynames.drop("dr_ea_count", 1)
babynames.head(5)

,State,Sex,Year,Name,Count
76150,NY,F,1978,Deandrea,5
95305,NY,F,1988,Deandrea,6
120981,NY,F,1999,Deandrea,5
92724,NY,F,1987,Deandra,10
233222,NY,M,1979,Andreas,15


### Approach 2: Generate an index sorted in the desired order

In [37]:
#let's start over by first scrambling the order of babynames
babynames = babynames.sample(frac=1) # generating 100% sample of dataframe
babynames.head(5)

,State,Sex,Year,Name,Count
25516,NY,F,1944,Annabelle,17
302380,NY,M,2018,Juelz,13
174523,NY,F,2021,Rosemary,32
1877,NY,F,1913,Sabina,8
20324,NY,F,1938,Mona,40


Another approach is to take advantage of the fact that .loc can accept an index. That is:
 + df.loc[idx] returns df with its rows in the same order as the given index.
 + Only works if the index exactly matches the DataFrame.

The first step was to create a sequence of the lengths of the names.

In [38]:
# Create Series of only the lengths of the names

name_lengths = babynames["Name"].str.len() #####
name_lengths.head(5)

25516     9
302380    5
174523    8
1877      6
20324     4
Name: Name, dtype: int64

The next step is to sort the new series we just created.

In [39]:
# Sort the series of only name lengths

name_lengths_sorted_by_length = name_lengths.sort_values(ascending=True)
name_lengths_sorted_by_length.head(5)

245816    2
41433     2
138124    2
206514    2
264895    2
Name: Name, dtype: int64

Next, we pass the index of the sorted series to the loc method of the original dataframe.

In [40]:
# Save the index of name_lengths_sorted_by_length

index_sorted_by_length = name_lengths_sorted_by_length.index
index_sorted_by_length

Int64Index([245816,  41433, 138124, 206514, 264895, 225919, 135301, 250334,
            221177, 242714,
            ...
             93081,  58087,  45464, 250375, 266621,  93527, 113868, 288353,
             51703, 250865],
           dtype='int64', length=309532)

In [41]:
babynames.loc[index_sorted_by_length].head(5)

,State,Sex,Year,Name,Count
245816,NY,M,1988,Bo,5
41433,NY,F,1958,Jo,590
138124,NY,F,2006,Zi,6
206514,NY,M,1955,Al,20
264895,NY,M,1999,Ho,5


Note we can also do this all in one line:

In [42]:
# Do this in one line

babynames.loc[babynames["Name"].str.len().sort_values(ascending=True).index].head(5)

,State,Sex,Year,Name,Count
245816,NY,M,1988,Bo,5
41433,NY,F,1958,Jo,590
138124,NY,F,2006,Zi,6
206514,NY,M,1955,Al,20
264895,NY,M,1999,Ho,5


# An Overview of Pandas GroupBy

In [43]:
df = pd.read_csv("elections.csv")
df

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
2,Anderson,Independent,6.6,1980,loss
...,...,...,...,...,...
20,Romney,Republican,47.2,2012,loss
21,Clinton,Democratic,48.2,2016,loss
22,Trump,Republican,46.1,2016,win


## Grouping a Series by a Series

Let's group the `%` Series by the `Party` Series. A call to [`groupby`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) does that, but what is the object that results?

In [44]:
df['%'].groupby(df['Party'])

In [45]:
percent_grouped_by_party = df['%'].groupby(df['Party'])
type(percent_grouped_by_party)

pandas.core.groupby.generic.SeriesGroupBy

As we see, `percent_grouped_by_party` is **NOT** a DataFrame, Series, or Index. Instead, it's a `SeriesGroupBy` object. A `SeriesGroupBy` consists of `groups`, one for each of the distinct values of the `Party` column. If we ask to see these groups, we'll be able to see which indices in the original DataFrame correspond to each group.

In [46]:
percent_grouped_by_party.groups

{'Democratic': [1, 4, 6, 7, 10, 13, 15, 17, 19, 21], 'Independent': [2, 9, 12], 'Republican': [0, 3, 5, 8, 11, 14, 16, 18, 20, 22]}

The `percent_grouped_by_party` object is capable of making computations across all these groups. For example, if we call the `mean` method of the `SeriesGroupBy` class, we'll get a new `Series` containing the mean of the "Democratic" `Series`, the mean of the "Independent" `Series`, and the mean of the "Republican" `Series`.

In [47]:
# Find the average % by 'Party'

percent_grouped_by_party.mean()

Party
Democratic     46.53
Independent    11.30
Republican     47.86
Name: %, dtype: float64

The output of the `mean` methood is a regular ole pandas Series.

In [48]:
type(percent_grouped_by_party.mean())

pandas.core.series.Series

`SeriesGroupBy` objects have many other handy methods, e.g. max and min.

In [49]:
# Find max values

percent_grouped_by_party.max()

Party
Democratic     52.9
Independent    18.9
Republican     58.8
Name: %, dtype: float64

In [50]:
# Find min values

percent_grouped_by_party.min()

Party
Democratic     37.6
Independent     6.6
Republican     37.4
Name: %, dtype: float64

In [51]:
# Find size of each group

percent_grouped_by_party.size()

Party
Democratic     10
Independent     3
Republican     10
Name: %, dtype: int64

In [52]:
# Find first values

percent_grouped_by_party.first()

Party
Democratic     41.0
Independent     6.6
Republican     50.7
Name: %, dtype: float64

We can iterate over a `SeriesGroupBy` object. Each element is a pair of `(name, group)`, where `name` is a String label for the group, and `group` is a `Series` corresponding to all the values from that group.

In [53]:
from IPython.display import display  # like print, but for complex objects

for name, group in percent_grouped_by_party:
    print('Name:', name)
    print(type(group))
    display(group.head())
    print('\n')

Name: Democratic
<class 'pandas.core.series.Series'>


1     41.0
4     37.6
6     45.6
7     43.0
10    49.2
Name: %, dtype: float64



Name: Independent
<class 'pandas.core.series.Series'>


2      6.6
9     18.9
12     8.4
Name: %, dtype: float64



Name: Republican
<class 'pandas.core.series.Series'>


0     50.7
3     58.8
5     53.4
8     37.4
11    40.7
Name: %, dtype: float64

## Grouping a Series by Multiple Series

We can also group a Series by multiple Series. For example, suppose we want to track all combinations of `{'Democratic', 'Republican', and 'Independent'}` and `{'win', 'loss'}`. 

In [54]:
df

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
2,Anderson,Independent,6.6,1980,loss
...,...,...,...,...,...
20,Romney,Republican,47.2,2012,loss
21,Clinton,Democratic,48.2,2016,loss
22,Trump,Republican,46.1,2016,win


In [55]:
percent_grouped_by_party_and_result = df['%'].groupby([df['Party'], df['Result']])
percent_grouped_by_party_and_result.groups

{('Democratic', 'loss'): [1, 4, 6, 13, 15, 21], ('Democratic', 'win'): [7, 10, 17, 19], ('Independent', 'loss'): [2, 9, 12], ('Republican', 'loss'): [8, 11, 18, 20], ('Republican', 'win'): [0, 3, 5, 14, 16, 22]}

Given this groupby object, we can compute the average percentage earned every time each of the parties won and lost the presidential election. We see that at least between 1980 and 2016, the Republicans have typically lost and won their elections by wider margins.

In [56]:
# Find the mean

party_result_mean = percent_grouped_by_party_and_result.mean()
party_result_mean

Party        Result
Democratic   loss      44.85
             win       49.05
Independent  loss      11.30
Republican   loss      42.75
             win       51.27
Name: %, dtype: float64

The careful reader will note that the returned object looks a little funny. It seems to have two indexes! If we check the type of this object, we'll see it's just a regular Series.

In [57]:
type(party_result_mean)

pandas.core.series.Series

However if we request to see the index of this `Series`, we see that it is a "MultiIndex", which is a special type of index used for data that is indexed in two or more ways.

In [58]:
party_result_mean.index

MultiIndex([( 'Democratic', 'loss'),
            ( 'Democratic',  'win'),
            ('Independent', 'loss'),
            ( 'Republican', 'loss'),
            ( 'Republican',  'win')],
           names=['Party', 'Result'])

Selecting an element of a multi-index gives you a series with just an index.

In [59]:
party_result_mean['Republican']

Result
loss    42.75
win     51.27
Name: %, dtype: float64

Or you can select both levels at once.

In [60]:
party_result_mean['Democratic', 'win']

49.05

Or you can slice.

In [61]:
party_result_mean[:, "loss"]

Party
Democratic     44.85
Independent    11.30
Republican     42.75
Name: %, dtype: float64

## Grouping a DataFrame by a Series

We can also group an entire dataframe by one or more Series. The result is a `DataFrameGroupBy` object:

In [62]:
everything_grouped_by_party = df.groupby('Party')
everything_grouped_by_party

As in our previous example, this object contains three `group` objects, one for each party label.

In [63]:
everything_grouped_by_party.groups

{'Democratic': [1, 4, 6, 7, 10, 13, 15, 17, 19, 21], 'Independent': [2, 9, 12], 'Republican': [0, 3, 5, 8, 11, 14, 16, 18, 20, 22]}

Just as with `SeriesGroupBy` objects, we can iterate over a `DataFrameGroupBy` object to understand what is effectively inside.

In [64]:
for n, g in everything_grouped_by_party:
    print('name:', n)
    display(g.head())

name: Democratic


,Candidate,Party,%,Year,Result
1,Carter,Democratic,41.0,1980,loss
4,Mondale,Democratic,37.6,1984,loss
6,Dukakis,Democratic,45.6,1988,loss
7,Clinton,Democratic,43.0,1992,win
10,Clinton,Democratic,49.2,1996,win


name: Independent


,Candidate,Party,%,Year,Result
2,Anderson,Independent,6.6,1980,loss
9,Perot,Independent,18.9,1992,loss
12,Perot,Independent,8.4,1996,loss


name: Republican


,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
3,Reagan,Republican,58.8,1984,win
5,Bush,Republican,53.4,1988,win
8,Bush,Republican,37.4,1992,loss
11,Dole,Republican,40.7,1996,loss


And just like `SeriesGroupBy` objects, we can apply methods like `mean` to compute the mean of each group. Since a `DataFrameGroupBy` is linked to the entire original dataframe (instead of to a single column from the dataframe), we calculate a mean for every numerical column. In this example below, we get the mean vote earned (as before), and the mean year (which isn't a useful quantity).

In [65]:
everything_grouped_by_party.mean()

,%,Year
Party,,
Democratic,46.53,1998.00
Independent,11.30,1989.33
Republican,47.86,1998.00


Where did all the other columns go in the mean above? They are *nuisance columns*, which get automatically eliminated from an operation where it doesn't make sense (such as a numerical mean).

## Grouping a DataFrame by Multiple Series

DataFrames may also be grouped by multiple series at once. For example, we can repeat what we did with a Series above and group the entire DataFrame by Party and Result. After aggregation, we end up with a DataFrame that has a MultiIndex.

In [66]:
everything_grouped_by_party_and_result = df.groupby([df['Party'], df['Result']])

In [67]:
everything_grouped_by_party_and_result.mean()

%     Year
Party       Result                
Democratic  loss    44.85  1995.33
            win     49.05  2002.00
Independent loss    11.30  1989.33
Republican  loss    42.75  2002.00
            win     51.27  1995.33

In [68]:
df[df["Party"] == "Democratic"]

,Candidate,Party,%,Year,Result
1,Carter,Democratic,41.0,1980,loss
4,Mondale,Democratic,37.6,1984,loss
6,Dukakis,Democratic,45.6,1988,loss
...,...,...,...,...,...
17,Obama,Democratic,52.9,2008,win
19,Obama,Democratic,51.1,2012,win
21,Clinton,Democratic,48.2,2016,loss


## Pivot Tables

Recall from before that we were able to group the % Series by the "Party" and "Result" Series, allowing us to understand the average vote earned by each party under each election result.

In [69]:
percent_grouped_by_party_and_result = df['%'].groupby([df['Party'], df['Result']])
percent_grouped_by_party_and_result.mean()

Party        Result
Democratic   loss      44.85
             win       49.05
Independent  loss      11.30
Republican   loss      42.75
             win       51.27
Name: %, dtype: float64

Because we called `groupby` on a Series, the result of our aggregation operation was also a Series. However, I believe this data is more naturally expressed in a tabular format, with Party as the rows, and Result as the columns. The `pivot_table` operation is the natural way to achieve this data format.

In [70]:
df_pivot = df.pivot_table(
    index='Party', # the rows (turned into index)
    columns='Result', # the column values
    values='%', # the field(s) to processed in each group
    aggfunc=np.mean, # group operation
)
df_pivot.head()

Result,loss,win
Party,,
Democratic,44.85,49.05
Independent,11.30,NaN
Republican,42.75,51.27


The basic idea is that you specify a Series to be the `index` (i.e. rows) and a Series to be the `columns`. The data in the specified `values` is then grouped by all possible combinations of values that occur in the `index` and `columns` Series. These groups are then aggregated using the `aggfunc`, and arranged into a table that matches the requested `index` and `columns`. The diagram below summarizes how pivot tables are formed. 

![groupby](pivot_table_overview.png)

For more on pivot tables, see [this excellent tutorial](http://pbpython.com/pandas-pivot-table-explained.html) by Chris Moffitt.

## Using Groups to Filter Datasets

In [71]:
everything_grouped_by_year = df.groupby('Year')

In [72]:
everything_grouped_by_year.filter(lambda subframe: subframe["%"].sum() < 97)

,Candidate,Party,%,Year,Result
3,Reagan,Republican,58.8,1984,win
4,Mondale,Democratic,37.6,1984,loss
13,Gore,Democratic,48.4,2000,loss
14,Bush,Republican,47.9,2000,win
21,Clinton,Democratic,48.2,2016,loss
22,Trump,Republican,46.1,2016,win


A visual picture of how filtering works is shown below.

![filter](filter_overview.png)

## Using isin for filtering

This isn't actually related to `groupby` at all, but it seemed natural to bring it up here. 

Sometimes we want to restrict our attention to only rows where certain values appear. For example, we saw last time how we'd look at only rows that contain "Democratic" candidates.

In [73]:
df[df["Party"] == "Democratic"]

,Candidate,Party,%,Year,Result
1,Carter,Democratic,41.0,1980,loss
4,Mondale,Democratic,37.6,1984,loss
6,Dukakis,Democratic,45.6,1988,loss
...,...,...,...,...,...
17,Obama,Democratic,52.9,2008,win
19,Obama,Democratic,51.1,2012,win
21,Clinton,Democratic,48.2,2016,loss


Suppose we wanted to filter such that all Republicans and Democrats appeared. One ugly way to do this would be `df[(df["Party"] == "Democratic") | (df["Party"] == "Republican")]`. However, a better idea is to use the `isin` method.

In [74]:
df[(df["Party"] == "Democratic") | (df["Party"] == "Republican")]

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
3,Reagan,Republican,58.8,1984,win
...,...,...,...,...,...
20,Romney,Republican,47.2,2012,loss
21,Clinton,Democratic,48.2,2016,loss
22,Trump,Republican,46.1,2016,win


In [75]:
# do this with .isin method

df[df["Party"].isin(["Democratic", "Republican"])]

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
3,Reagan,Republican,58.8,1984,win
...,...,...,...,...,...
20,Romney,Republican,47.2,2012,loss
21,Clinton,Democratic,48.2,2016,loss
22,Trump,Republican,46.1,2016,win


## List Arguments to pivot_table (Extra)

The arguments to our pivot_table method can also be lists. A few examples are given below.

If we pivot such that only our `columns` argument is a list, we end up with columns that are MultiIndexed.

In [76]:
df.pivot_table(
    index='Result', # the rows (turned into index)
    columns=['Party', 'Candidate'], # the column values
    values='Year', # the field(s) to processed in each group
    aggfunc=list, # group operation
)

Party     Democratic                                ... Republican  \
Candidate     Carter       Clinton Dukakis    Gore  ...     McCain   
Result                                              ...              
loss          [1980]        [2016]  [1988]  [2000]  ...     [2008]   
win              NaN  [1992, 1996]     NaN     NaN  ...        NaN   

Party                                    
Candidate        Reagan  Romney   Trump  
Result                                   
loss                NaN  [2012]     NaN  
win        [1980, 1984]     NaN  [2016]  

[2 rows x 15 columns]

If we pivot such that only our `index` argument is a list, we end up with rows that are MultiIndexed.

In [77]:
df.pivot_table(
    index=['Party', 'Candidate'], # the rows (turned into index)
    columns='Result',# the column values
    values='%', # the field(s) to processed in each group
    aggfunc=np.mean, # group operation
)

Result                loss    win
Party      Candidate             
Democratic Carter     41.0    NaN
           Clinton    48.2  46.10
           Dukakis    45.6    NaN
...                    ...    ...
Republican Reagan      NaN  54.75
           Romney     47.2    NaN
           Trump       NaN  46.10

[15 rows x 2 columns]

If we pivot such that only our values argument is a list, then we again get a DataFrame with multi-indexed Columns.

In [78]:
df.pivot_table(
    index='Party', # the rows (turned into index)
    columns='Result',# the column values
    values=['%', 'Year'], # the field(s) to processed in each group
    aggfunc=np.mean, # group operation
)

%            Year         
Result        loss    win     loss      win
Party                                      
Democratic   44.85  49.05  1995.33  2002.00
Independent  11.30    NaN  1989.33      NaN
Republican   42.75  51.27  2002.00  1995.33

## Custom Aggregation Functions

As described above, both `SeriesGroupBy` and `DataFrameGroupBy` objects have lots of handy methods for computing aggregate values for groups, e.g.

In [79]:
percent_grouped_by_party.min()

Party
Democratic     37.6
Independent     6.6
Republican     37.4
Name: %, dtype: float64

In [80]:
everything_grouped_by_party.median()

,%,Year
Party,,
Democratic,48.25,1998
Independent,8.40,1992
Republican,47.55,1998


It turns out that all of these GroupBy methods are just shorthand for a more powerful and universal method of our GroupBy objects called agg. For example, `.min()` is just shorthand for `.agg(min)`, where `min` refers to the function `min`.

In [81]:
percent_grouped_by_party.agg(sum)

Party
Democratic     465.3
Independent     33.9
Republican     478.6
Name: %, dtype: float64

Naturally, we can define our own custom aggregation functions. For example, the function below returns the first item in a series.

In [82]:
def average_of_first_and_last(series):
    return (series.iloc[0] + series.iloc[-1])/2

We can supply this function as a custom aggregation function for each series. As you can see, nuisance columns are automatically removed.

In [83]:
percent_grouped_by_party_and_result.agg(average_of_first_and_last)

Party        Result
Democratic   loss      44.60
             win       47.05
Independent  loss       7.50
Republican   loss      42.30
             win       48.40
Name: %, dtype: float64

`agg` is fundamental to our use of GroupBy objects. Indeed, you will rarely call `groupby` without also calling `agg`, at least implicitly. We can summarize the grouping process with the following diagram, inspired by a similar diagram created by DS100 instructor Joey Gonzales. Diagram source at [this link](https://docs.google.com/presentation/d/1FrYg6yd6B-CIgfWLWm4W8vBhfmJ6Qt9dKkN-mlN5AKU/edit#slide=id.g4131093782_0_40).

![groupby](groupby_overview.png) 

The result of calling `groupby` then `agg` on a `Series` is also a `Series`, and the result of calling `groupby` then `agg` on a `DataFrame` is also typically a `DataFrame`, though there are exceptions, e.g. if you use the aggregation function `size`, you will get back a `Series`.

In [84]:
def list_of_first_and_last(series):
    return [series.iloc[0], series.iloc[-1]]

In [85]:
everything_grouped_by_party.agg(list_of_first_and_last)

,Candidate,%,Year,Result
Party,,,,
Democratic,"[Carter, Clinton]","[41.0, 48.2]","[1980, 2016]","[loss, loss]"
Independent,"[Anderson, Perot]","[6.6, 8.4]","[1980, 1996]","[loss, loss]"
Republican,"[Reagan, Trump]","[50.7, 46.1]","[1980, 2016]","[win, win]"
